In [ ]:
!pip install langchain_community langchain_huggingface sentence_transformers

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

import os
from tqdm import tqdm
import ollama
import pandas as pd
import pdfplumber
from collections import defaultdict
import itertools
import re
from langchain_core.documents import Document
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
model_name = "BAAI/bge-m3"
if not os.path.exists(model_name):
    model = SentenceTransformer(model_name)
    model.save(model_name)

In [ ]:
embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={'device': "mps"},
        encode_kwargs={'normalize_embeddings': True},
    )

In [ ]:
emb_vectors = embeddings.embed_documents([
    "안녕하세요.",
    "반갑습니다.",
    "감사합니다.",
])
print(emb_vectors[0][:4])
print(emb_vectors[1][:4])
print(emb_vectors[2][:4])

In [ ]:
len(emb_vectors[0])

In [ ]:
chunking_data = ""

In [ ]:
vector_store = FAISS.from_documents(chunking_data, embeddings)

In [ ]:
diagnosis_str = ""

In [ ]:
medical_info = defaultdict(list)
medical_info["hospital_1"] = (
    '질병 또는 부상명: (S52590) 요골 하단의 상세불명 골절, 폐쇄성, '
    '(S62630) 기타 손가락의 중지골 골절, 폐쇄성, '
    '(S836) 무릎의 기타 및 상세불명 부분의 염좌 및 긴장, '
    '(M2416) 기타 관절연골장애, 무릎관절, '
    '(M2406) 관절안의 유리체 무릎관절, '
    '(M6586) 기타 윤활막염 및 힘줄윤활막염 무릎관절. '
    '\n치료기간: 입원 2022년 11월 26일부터 2022년 12월 24일까지(29 일간). '
    '\n소견서 내용: 상기환자는 2022/11/1 넘어져 수상후 타병원 진료후 내원하신 환자로 '
    '우측 요골부, 우측 제4 수지부의 골절 진단과 우측 무릎의 통증으로 '
    '안정가료 및 통증완화에 대한 치료를 위하여 상기 기간 동안 입원치료하였으며 '
    '추후 우측 손목, 무릎의 지속적인 관찰 및 재활치료가 필요 할 것으로 사료됨. '
    '\n의료기관 명칭: 사각 종합병원\n\n'
)

medical_info["hospital_2"] = (
    '질병 또는 부상명: (M2416) 기타 관절연골장애 아래다리, '
    '(M2406) 관절안의 유리체, 아래다리, '
    '(M6586) 기타 윤활막염 및 힘줄윤활막염 아래다리, '
    '(M170) 앙쪽 원발성 무릎관절증, '
    '(S52590) 요골 하단의 상세불명 골절, 폐쇄성, ' 
    '(S62630) 기타 손가락의 중지골 골절, 폐쇄성. '
    '\n치료기간: 입원 2022년 12월 24일부터 2023년 01월 10일까지(18 일간). '
    '\n소견서 내용: 상기환자 양측 무릎과 우측 손목, 우측 네번째 손가락 통증으로 입원한 환자로 '
    '이학적 검사 및 단순 방사선 검사상 상기 병명으로 진단되었으며 '
    'VAS 7의 무릎 통중과 우측 손목 손가락 골절로 인한 ROM 제한으로 증상 호전시끼지 '
    '물리치료, 도수치료, 체외 충격파 치료 등 보존적 치료가 필요할 것으로 판단됨 '
    '상기 소견은 초진 소견이며, 추후 경과에 따라 재평가 요함.'
    '\n구두소견: 사각 병원에서 치료 후에 전원 온 환자로 입원 경위에 대해서는, '
    '골절, 관절 내 유리체, 퇴행성 관절염, 강직 등 단순 통증으로 내원한 것 외에는, '
    '더 이상 드릴 답변 없음. '
    '필요시 더 입원을 할 수도 있는 환자이고 심평원 적정 의료 기준에 따라 퇴원시킨 것으로, '
    '적정입원기간을 명확하게 산정하기 어려움.'
    '\n의료기관 명칭: 서울 창업허브 종합병원\n\n'
)

medical_info["interview_1"] = (
    '고객 안내일자: 2023-02-08.'
    '\n고객 안내내용: 현장심사 안내.'
    '\n고객 반응: 문답서 작성과 면담을 거부함.'
)

medical_info["interview_2"] = (
    '고객 안내일자: 2023-03-17.'
    '\n고객 안내내용: 최초 내원경위로 2022.11.1에 넘어지고 나서, '
    '요골 하단의 상세불명 골절, 폐쇄성 진단으로 주병명이 확인되어, '
    '재해로 검토되어질 수 있음을 안내.'
    '\n고객 반응: 넘어져서 내원한 것은 맞으나, '
    '어깨, 무릎 등은 원래부터 가지고 있는 질병으로 인해 입원치료를 받은 것이니, '
    '질병으로 처리됨이 타당함.'
)

values_list = list(medical_info.values())
result_string = ' '.join(values_list)

In [ ]:
messages = [
    {"role": "system", "content": "너는 전문성이 매우 높은 수준의 손해사정보고서 작성 챗봇이야. 전문적인 용어로 답변해"},
    {"role": "user", "content": "아래의 조사기록을 분석해서 손해사정보고서 작성" + result_string},
]

result_stream = ollama.chat(
      model="aya:8b",
      messages=messages,
      stream=True
    )

In [ ]:
contents_output = ''
for chunk in result_stream:
    content = chunk['message']['content']
    print(chunk['message']['content'], end='', flush=True)
    contents_output += content
print('\n')

In [ ]:
relevant_docs = vector_store.similarity_search(result_string, k=3)

In [ ]:
insurance_context = "\n".join(doc.page_content for doc in relevant_docs)

In [ ]:
insurance_context

In [ ]:
input_prompt = (
        # f'Context: {insurance_context} \n '
        f'Question: 위의 데이터를 바탕으로 다음의 손해사정보고서에 기록된 질병 또는 부상에 대한 보험금 지급 여부 및 예상금액 판단 \n'
        # f' {contents_output}'
    )

In [ ]:
messages = [
    {"role": "system", "content": "너는 전문성이 매우 높은 수준의 손해사정보고서 작성 챗봇이야. 전문적인 용어로 답변해"},
    {"role": "user", "content": "아래의 조사기록을 분석해서 손해사정보고서 작성" + result_string},
    {"role": "assistant", "content": contents_output},
    {"role": "user", "content": input_prompt},
]

In [ ]:
result_stream = ollama.chat(
      model="aya:8b",
      messages=messages,
      stream=True
    )

for chunk in result_stream:
    content = chunk['message']['content']
    print(chunk['message']['content'], end='', flush=True)
print('\n')